In [0]:
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("db", "")
dbutils.widgets.text("model", "")
dbutils.widgets.text("run_id", "")
dbutils.widgets.text("user", "")

In [0]:
model_class = "global" if "NeuralForecast" in dbutils.widgets.get("model") else "foundation" 
%pip install -r ../requirements-{model_class}.txt --quiet
dbutils.library.restartPython()

In [0]:
catalog = dbutils.widgets.get("catalog")
db = dbutils.widgets.get("db")
model = dbutils.widgets.get("model")
run_id = dbutils.widgets.get("run_id")
user = dbutils.widgets.get("user")

In [0]:
from mmf_sa import run_forecast
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)
logging.getLogger("py4j.clientserver").setLevel(logging.ERROR)


run_forecast(
    spark=spark,
    train_data=f"{catalog}.{db}.m4_weekly_train",
    scoring_data=f"{catalog}.{db}.m4_weekly_train",
    scoring_output=f"{catalog}.{db}.weekly_scoring_output",
    evaluation_output=f"{catalog}.{db}.weekly_evaluation_output",
    model_output=f"{catalog}.{db}",
    group_id="unique_id",
    date_col="ds",
    target="y",
    freq="W",
    prediction_length=4,
    backtest_length=12,
    stride=1,
    metric="smape",
    train_predict_ratio=1,
    data_quality_check=True,
    resample=False,
    active_models=[model],
    experiment_path=f"/Users/{user}/mmf/m4_weekly",
    use_case_name="m4_weekly",
    run_id=run_id,
    accelerator="gpu",
)